<a href="https://colab.research.google.com/github/avlis-MMO/Deep_Learning/blob/main/Project_6_Natural_Language_Processing/RNN_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install numpy
!pip install matplotlib
!pip install scipy
!pip install pandas

# **Load Data**

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd

data = pd.read_csv("../content/drive/MyDrive/archive/IMDB_Dataset.csv")

# **Preprocess**

In [ ]:


data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})

x_train, y_train = data['review'][:40000].values.tolist(),  data['sentiment'][:40000].values.tolist()
x_test, y_test = data['review'][40000:50000].values.tolist(), data['sentiment'][40000:50000].values.tolist()

y_train = np.array(y_train)
y_test = np.array(y_test)

print(x_train[0], y_train[1])

# **Tokenization**

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(data['review'].values.tolist())

idx = tokenizer.word_index

tokens_train = tokenizer.texts_to_sequences(x_train)
tokens_test = tokenizer.texts_to_sequences(x_test)
#print(tokens_train[0])

length = []
for sentences in tokens_train:
  length.append(len(sentences))
print(sum(length)/len(length))

num_tokens = [len(tokens) for tokens in tokens_train + tokens_test]
num_tokens = np.array(num_tokens)

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)

tokens_train = tf.keras.utils.pad_sequences(tokens_train, max_tokens)
tokens_test = tf.keras.utils.pad_sequences(tokens_test, max_tokens)

# **Build Model**

In [ ]:
embedding_dim = 32
lstm_units = 128

model = model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(10000, embedding_dim),
    tf.keras.layers.LSTM(lstm_units),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# **Compile Model**

In [ ]:
learning_rate = 1e-5
optimizer = tf.keras.optimizers.Adam(learning_rate)

model.compile(optimizer = optimizer, loss='binary_crossentropy', metrics=['accuracy'])


# **Train**

In [ ]:
history = model.fit(tokens_train, y_train, epochs=10, validation_data=(tokens_test, y_test))


In [ ]:
def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [idx[word] if word in idx else 0 for word in tokens]
  return tf.keras.utils.pad_sequences([tokens], max_tokens)[0]

text = 'This movie was awful, dont understand why anybody would want to see it'
encoded = encode_text(text)


In [ ]:
def predict(text):
  encoded_text=encode_text(text)
  pred = np.zeros((1,max_tokens))
  pred[0] = encoded_text
  results=model.predict(pred)
  print(results[0])

text = "I hated this movie, there wasn't anything good about it. Waste of time"
encoded = predict(text)

text = 'I love this movie it was very good, i recommend it to everyone'
encoded = predict(text)